In [1]:
install.packages("cluster")
install.packages("factoextra")
install.packages("fpc")
install.packages("dbscan")
library(cluster)
library(factoextra)
library("fpc")
library("dbscan")

package 'cluster' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\vyas0\AppData\Local\Temp\RtmpERIySs\downloaded_packages
package 'factoextra' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\vyas0\AppData\Local\Temp\RtmpERIySs\downloaded_packages
package 'fpc' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\vyas0\AppData\Local\Temp\RtmpERIySs\downloaded_packages
package 'dbscan' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\vyas0\AppData\Local\Temp\RtmpERIySs\downloaded_packages


Loading required package: ggplot2
Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Welcome! Related Books: `Practical Guide To Cluster Analysis in R` at https://goo.gl/13EFCZ

Attaching package: 'dbscan'

The following object is masked from 'package:fpc':

    dbscan



In [2]:
complex8 <- read.csv("Complex8.data", header=FALSE)
dataset <- read.csv("Pima.csv", header = FALSE)

# 0.

In [3]:
# 0. Transform the Pima dataset into a dataset ZPima by z-scoring the first 8 attributes of the dataset, and copying the 
# 9th attribute of the dataset 
ZPima <- scale(dataset[,1:8])
ZPima <- cbind(ZPima, dataset[,9])

# 1.

In [4]:
# 1. Write an R-function entropy(a,b) that computes the entropy and the pecentage of outliers of a clustering 
# result based on an apriori given set of class lables

a1 <- c(0,1,1,1,1,2,2,3)
a2 <- c('1','1','1','0','0','2','2','2')
b <- c('A','A','A','E','E','D','D','C')

entropyvec <- function(vec) {
  s <- sum(vec)
 
  totalh <- Reduce(function(acc, elem) {
    prob <- elem / s
    h <- if (prob == 0) 0 else prob * log2(prob)
    return (acc - h)
  }, vec, 0)
 
  return (totalh)
}

entropy <- function(clusterassignment, groundtruth) {
  clusterlevels <- clusterassignment
  clusterclass <- table(clusterassignment, groundtruth)

  n <- nrow(clusterclass)
  entropy_weight <- array(dim=c(n, 2))
  population <- 0
 
  for (i in 1:n) {
    clustersize <- sum(clusterclass[i,])
    population <- population + clustersize
    entropy_weight[i,] = c(entropyvec(clusterclass[i,]), clustersize)
  }
  percentage <- 0
  outliers <- sum(clusterclass[1,])
  entropy_weight[1,] = c(0, 0)
  percentage <- outliers / population
  population <- population - outliers
 
  totalh <- 0
  if (population > 0) {
    for (i in 1:n) {
      totalh <- totalh + entropy_weight[i, 1] * entropy_weight[i, 2] / population
    }
  }
 
  return (c(totalh, percentage))
}
print(entropy(a1, b))
print(entropy(a2, b))

[1] 0.5714286 0.1250000
[1] 0.4591479 0.2500000


# 2.

In [5]:
# 2. Write an R-function wabs-dist(u,v,w) that takes two vectors u, v as an input and computes the distance 
# between u and v as the sum of the absolute weighted differences—using the weights in w .*
wabs_dist <- function(u,v,w){
   return(sum((abs(u-v))*w))
} 

wabs_dist(c(1,2), c(4,5), c(0.2,0.3))
wabs_dist(c(4,5),c(9,12), c(0.2,0.3))

[1] 1.5

[1] 3.1

# 3.

In [6]:
# 3. Write an R-function create-dm(x,w) that returns a distance matrix for the objects in dataframe x by calling the 
# wabs-dist(a,b,w) for all pairs of objects a and b belonging to x. **
create_dm <- function(x,w){
    rows <- nrow(x)
    data <- matrix(nrow = rows, ncol = rows, data = 0)
    for(i in 1:rows){
        for(j in 1:rows){
            data[i,j] <- wabs_dist(as.numeric(as.vector(x[i,])), as.numeric(as.vector(x[j,])), w)
            data[j,i]=data[i,j];
        }
        data[i,i] = 0
    }
    
    return(data)
}

create_dm(data.frame("x" = c(1,4,9), "y" = c(2,5,12)), c(0.2,0.3))



0.0,1.5,4.6
1.5,0.0,3.1
4.6,3.1,0.0


# 4.

In [7]:
# 4. Run K-means on the ZPima dataset for k=6 and k=9 and nstart=20; 
# next run PAM for k=6 with distance matrices that have been created using the following weight vectors for the 
# ZPima dataset using the function create-dm─obtaining 3 different PAM clustering results:
# a. (1,1,1,1,1,1,1,1)
# b. (0.2,1,0,0,0,1,0.2,1)
# c. (0,1,0,0,0,1,0,0)
# For the 5 obtained clustering results report the overall entropy, the entropy of each cluster, 
# the majority class and the centroid/medoid of each cluster. 
# Next, visualize the clustering result of the K-means run for k=6/9 and for the 3 PAM results in the 
# Plasma glucose/Body mass index (Attribute 2&6 Space) for the original dataset . 
# Interpret the obtained results! Does changing the distance metric affect the PAM results? Do the results tell
# you anything which attributes are important for diagnosing diabetes, and about the difficulty diagnosing diabetes?

k6 <- kmeans(x = ZPima[,1:8], centers = 6, nstart = 20)
k9 <- kmeans(x = ZPima[,1:8], centers = 9, nstart = 20)

pA <- create_dm(as.data.frame(ZPima[,1:8]), c(1,1,1,1,1,1,1,1))
pB <- create_dm(as.data.frame(ZPima[,1:8]), c(0.2,1,0,0,0,1,0.2,1))
pC <- create_dm(as.data.frame(ZPima[,1:8]), c(0,1,0,0,0,1,0,0))

In [ ]:
pamA <- pam(x = pA, k = 6, diss = TRUE)
pamB <- pam(x = pB, k = 6, diss = TRUE)
pamC <- pam(x = pC, k = 6, diss = TRUE)

# entropy(k6$cluster, ZPima[,9])[1]
# entropy(k9$cluster, ZPima[,9])[1]
# entropy(pamA$cluster, ZPima[,9])[1]
# entropy(pamB$cluster, ZPima[,9])[1]
# entropy(pamC$cluster, ZPima[,9])[1]

overall_entropy <- mean(c(entropy(k6$cluster, ZPima[,9])[1], entropy(k9$cluster, ZPima[,9])[1], entropy(pamA$clustering, ZPima[,9])[1], entropy(pamB$clustering, ZPima[,9])[1], entropy(pamC$clustering, ZPima[,9])[1]))

### The over all entropy is:

In [ ]:
overall_entropy

## K-means k = 6
### The entropy of the k-means model with k = 6 is  0.7210503. The majority class is in cluster 5 with 257 points.
### Centroid:

In [ ]:
k6$centers

## K-means k = 9
### The entropy of the k-means model with k = 6 is  0.7210503. The majority class is in cluster 5 with 170 points.
### Centroid:

In [ ]:
k9$centers

## PAM A  k = 6
### The entropy of the PAM model with k = 6 is  0.7210503. The majority class is in cluster 5 with 257 points.
### Medoids: 

In [ ]:
pamA$medoids

## PAM B k = 6
### The entropy of the PAM model with k = 6 is  0.7210503. The majority class is in cluster 5 with 257 points.
### 

In [ ]:
pamB$medoids

## PAM C k = 6
### The entropy of the PAM model with k = 6 is  0.7210503. The majority class is in cluster 5 with 257 points.
### 

In [ ]:
pamC$medoids

In [ ]:
newData <- dataset[,2]
# newData <- cbind(newData, dataset[,6])

pamA <- pam(x = pA, k = 6)
pamB <- pam(x = pB, k = 6)
pamC <- pam(x = pC, k = 6)

fviz_cluster(k6, data = as.data.frame(newData))
fviz_cluster(k9, data = as.data.frame(newData))
fviz_cluster(pamA)
fviz_cluster(pamB, data = newData)
fviz_cluster(pamC, data = newData)

pamA <- pam(x = pA, k = 6, diss = TRUE)
pamB <- pam(x = pB, k = 6, diss = TRUE)
pamC <- pam(x = pC, k = 6, diss = TRUE)

# 6.

In [ ]:
##### 6. Run DBSCAN on the ZPima dataset; try to find values for MinPoints and epsilon parameters, 
# such that the number of outliers is less than 15% and the number of clusters is between 2 and 12 ; 
# visualize the obtained clustering result in the Plasma glucose/Body mass index attribute Space on the original dataset 
# and report its entropy. Comment on the quality of the obtained clustering result. 

# dbscan::kNNdistplot(ZPima[,1:8], k=6)
# abline(h =1.9, lty = 2)
# db <- dbscan(ZPima[,1:8], eps = 2.1, minPts = 3)
db <- fpc::dbscan(ZPima[,1:8], eps = 1.9, MinPts = 3)

# Plot DBSCAN results
newData <- dataset[,2]
newData <- cbind(newData, dataset[,6])
fviz_cluster(db, as.data.frame(newData), geom = "point")

### Entropy:

In [ ]:
entropy(db$cluster, ZPima[, 9])[1]

# 7.

In [ ]:
# 7. Run K-means with nstart=20 for k=8 and k=11 for the Complex8 dataset; 
# visualize the results, compute their entropy, and discuss the obtained clustering results and its quality

ZComplex8 <- scale(complex8[,1:2])
ZComplex8 <- cbind(ZComplex8, complex8[,3])

complex8_k8 <- kmeans(ZComplex8[,1:2], 8, nstart = 20)
complex8_k11 <- kmeans(ZComplex8[,1:2], 11, nstart = 20)

cat("Entropy for k = 8: ", entropy(complex8_k8$cluster, complex8$V3), "\n")
fviz_cluster(complex8_k8, data = ZComplex8[,1:2])
cat("Entropy for k = 11: ", entropy(complex8_k11$cluster, complex8$V3))
fviz_cluster(complex8_k11, data = ZComplex8[,1:2])



# 8.

In [ ]:
#8. Next, run DBSCAN for the Complex8 dataset; try to find “good” parameter settings  
# for epsilon and Minpoints such that entropy of the obtained clustering result is maximized and the number of outliers is 
# less than 8% .  Develop a search procedure looks for the “best” DBSCAN clustering for the Complex8 dataset by varying 
# the epsilon/Minpoint parameters. Report the best clustering result you found, report the epsilon and Minpoint parameters 
# you used to obtain this clustering result and report  its percentage of outliers and entropy. 
# Results with higher entropy will receive higher scores; students who find the “best” result will get extra credit worse **’s!
# Also compare the DBSCAN results with those you obtained for K-means in Task 7, and assess which clustering algorithm did a 
# better job in clustering the Complex8 dataset! 

# dbscan::kNNdistplot(ZComplex8[,1:2], k=6)
# abline(h =.101, lty = 2)

db <- dbscan::dbscan(ZComplex8[,1:2], eps = .101, minPts = 4)
db
# # Plot DBSCAN results
# newData <- dataset[,2]
# newData <- cbind(newData, dataset[,6])
fviz_cluster(db, ZComplex8[,1:2], geom = "point")

### Entropy:

In [ ]:
entropy(db$cluster, ZComplex8[,3])[1]

## 9.